In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from maccabee.constants import Constants

In [3]:
_ = Constants.AxisNames.all()

{ 'ALIGNMENT': 'ALIGNMENT',
  'BALANCE': 'BALANCE',
  'OUTCOME_NONLINEARITY': 'OUTCOME_NONLINEARITY',
  'OVERLAP': 'OVERLAP',
  'PERCENT_TREATED': 'PERCENT_TREATED',
  'TE_HETEROGENEITY': 'TE_HETEROGENEITY',
  'TREATMENT_NONLINEARITY': 'TREATMENT_NONLINEARITY'}


In [6]:
class CausalModel():
    def __init__(self, dataset):
        self.dataset = dataset

    def fit(self):
        raise NotImplementedError

    def estimate_ITE(self):
        return 1

    def estimate_ATE(self):
        raise NotImplementedError

    def estimate(self, estimand, *args, **kwargs):
        if estimand == 'ATE':
            return self.estimate_ATE(*args, **kwargs)
        elif estimand == "ITE":
            return self.estimate_ITE(*args, **kwargs)
        else:
            raise UnknownEstimandException()

In [21]:
def a():
    pass

class Test():
    def a():
        

Test.a
# a.__get__(Test(), Test)

AttributeError: type object 'Test' has no attribute 'a'